# Ngrok을 통한 vllm Endpoint 제공 방법

## 개요

* colab 내에서 vllm을 통해 LLM 모델 실행 및 엔드포인트 실행
* 해당 엔드포인트를 외부에서 접근가능하도록 ngrok을 통해 외부 endpoint와 연결

## * Google Drive 연결 및 마운트

In [2]:
# 1. 구글 드라이브 연결
from google.colab import drive
import os

print("📂 구글 드라이브 연결 중...")
drive.mount('/content/drive')

# 2. 작업 디렉토리 변경 (서버의 cd 명령와 동일)
# 실제 프로젝트 폴더 경로로 이동
PROJECT_PATH = '/content/drive/MyDrive/ngrok_api_server'

if not os.path.exists(PROJECT_PATH):
    # 폴더가 없으면 생성 (최초 1회)
    os.makedirs(PROJECT_PATH)
    print(f"🆕 폴더 생성됨: {PROJECT_PATH}")
    print("⚠ 구글 드라이브에 가서 필요한 파일(train.py, config.ini 등)을 넣어주세요!")
else:
    os.chdir(PROJECT_PATH)
    print(f"✅ 작업 경로 이동 완료: {os.getcwd()}")

# 3. 현재 파일 목록 확인 (제대로 이동했는지 체크)
print("\n[현재 폴더 파일 목록]")
print(os.listdir())


📂 구글 드라이브 연결 중...
Mounted at /content/drive
🆕 폴더 생성됨: /content/drive/MyDrive/ngrok_api_server
⚠ 구글 드라이브에 가서 필요한 파일(train.py, config.ini 등)을 넣어주세요!

[현재 폴더 파일 목록]
['.config', 'drive', 'sample_data']


## * ngrok 및 vllm 패키지 설치

In [ ]:
!pip install vllm pyngrok

* 패키지 설치 후, 환경변수 설정 필요
  * NGROK_TOKEN
    * ngrok token을 키값으로 입력
    * https://dashboard.ngrok.com/get-started/your-authtoken

## * Ngrok Endpoint URL 생성

In [7]:
import os
from pyngrok import ngrok
from google.colab import userdata

NGROK_TOKEN = userdata.get('NGROK_TOKEN')
os.environ["NGROK_TOKEN"] = NGROK_TOKEN

ngrok.set_auth_token(NGROK_TOKEN)

public_url = ngrok.connect(8000).public_url
print(f"🚀 API 주소 생성 완료: {public_url}")
print(f"👉 이 주소를 복사해서 사용하세요: {public_url}/v1")

🚀 API 주소 생성 완료: https://unconstruable-nonfactually-cedric.ngrok-free.dev
👉 이 주소를 복사해서 사용하세요: https://unconstruable-nonfactually-cedric.ngrok-free.dev/v1


## * vllm 서버 실행

In [ ]:
!python -m vllm.entrypoints.openai.api_server \
  --model Qwen/Qwen3-4B-AWQ \
  --max-model-len 2048 \
  --gpu-memory-utilization 0.85 \
  --disable-log-stats \
  --enforce-eager \
  --max-num-seqs 1

2025-12-31 01:49:36.329890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767145776.361648    3940 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767145776.367769    3940 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767145776.383537    3940 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767145776.383567    3940 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767145776.383571    3940 computation_placer.cc:177] computation placer alr

(EngineCore_DP0 pid=4205) INFO 12-31 01:54:06 [core.py:259] init engine (profile, create kv cache, warmup model) took 161.81 seconds
(EngineCore_DP0 pid=4205) WARNING 12-31 01:54:08 [vllm.py:629] Inductor compilation was disabled by user settings,Optimizations settings that are only active duringInductor compilation will be ignored.
(EngineCore_DP0 pid=4205) INFO 12-31 01:54:08 [vllm.py:722] Cudagraph is disabled under eager mode
(APIServer pid=3940) INFO 12-31 01:54:08 [api_server.py:1099] Supported tasks: ['generate']
(APIServer pid=3940) WARNING 12-31 01:54:08 [model.py:1487] Default sampling parameters have been overridden by the model's Hugging Face generation config recommended from the model creator. If this is not intended, please relaunch vLLM instance with `--generation-config vllm`.
(APIServer pid=3940) INFO 12-31 01:54:08 [serving_responses.py:201] Using default chat sampling params from model: {'temperature': 0.6, 'top_k': 20, 'top_p': 0.95}
(APIServer pid=3940) INFO 12-31